## SQL Project - Sturtup on Apps for Book Lovers

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

### The Goal

The project goal is to generate a value proposition for a new product.

### Task

- Find the number of books released after January 1, 2000.
- Find the number of user reviews and the average rating for each book.
- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
- Identify the author with the highest average book rating (look only at books with at least 50 ratings).
- Find the average number of text reviews among users who rated more than 50 books.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Study the tables (print the first rows).

In [2]:
# study table "books" (print the first rows)
#books = pd.io.sql.read_sql("SELECT * FROM books LIMIT 10", con = engine)
books = pd.io.sql.read_sql("SELECT * FROM books", con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# study table "authors" (print the first rows)
authors = pd.io.sql.read_sql("SELECT * FROM authors", con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# study table "ratings" (print the first rows)
ratings = pd.io.sql.read_sql("SELECT * FROM ratings", con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
# study table "reviews" (print the first rows)
reviews = pd.io.sql.read_sql("SELECT * FROM reviews", con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
# # study table "publishers" (print the first rows)
publishers = pd.io.sql.read_sql("SELECT * FROM publishers", con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Make an SQL query for each of the tasks.

In [7]:
# Find the number of books released after January 1, 2000.
query1 = "SELECT COUNT(book_id) FROM books WHERE publication_date::date > '2000-01-01'::date"

In [8]:
# Find the number of user reviews and the average rating for each book.
query2 = '''
SELECT reviews.book_id, COUNT(review_id) AS "Number of reviews", AVG(rating) AS "Average rating" 
FROM ratings 
INNER JOIN reviews ON ratings.book_id = reviews.book_id GROUP BY reviews.book_id
'''

In [9]:
# Identify the publisher that has released the greatest number of books with more than 50 pages.
query3 = '''
SELECT publisher_id FROM (
SELECT COUNT(book_id) AS "bigbooks", publisher_id FROM books WHERE num_pages > 50 GROUP BY publisher_id
) AS temp ORDER BY bigbooks DESC LIMIT 1
'''

In [10]:
# Identify the author with the highest average book rating (look only at books with at least 50 ratings).
query4 = '''
SELECT authors.author --, books.author_id, books.book_id, COUNT(ratings.rating_id), AVG(ratings.rating) as Average 
FROM books 
INNER JOIN ratings ON books.book_id = ratings.book_id 
INNER JOIN authors ON books.author_id = authors.author_id 
GROUP BY books.book_id, authors.author
HAVING COUNT(ratings.rating_id) > 50 
ORDER BY AVG(ratings.rating) DESC 
LIMIT 1
'''

In [11]:
# Find the average number of text reviews among users who rated more than 50 books.
query5 = '''
SELECT AVG(temp.review_count) AS "Average review count" FROM (
    SELECT COUNT(review_id) AS review_count, username FROM reviews 
    WHERE username IN (
        SELECT username 
        FROM ratings 
        GROUP BY username 
        HAVING COUNT(rating_id) > 50
    ) GROUP BY username
) AS temp
'''

## Output the results of each query.

In [12]:
# Output the results for task: Find the number of books released after January 1, 2000.
pd.io.sql.read_sql(query1, con = engine)

,count
0,819


The number of books released after January 1, 2000 is 819 of 1000.

In [13]:
# Output the results for task: Find the number of user reviews and the average rating for each book.
aa = pd.io.sql.read_sql(query2, con = engine).sort_values(by='Average rating', ascending=False)
aa.describe()

,book_id,Number of reviews,Average rating
count,994.000000,994.00000,994.000000
mean,501.144869,26.32495,3.898699
std,288.845619,68.48706,0.562190
min,1.000000,2.00000,1.500000
25%,252.250000,4.00000,3.500000
50%,501.500000,9.00000,4.000000
75%,750.750000,18.00000,4.333333
max,1000.000000,1120.00000,5.000000


We find the number of user reviews and the average rating for each book.

In [14]:
# Output the results for task: Identify the publisher that has released the greatest number of books with more than 50 pages. 
pd.io.sql.read_sql(query3, con = engine)

,publisher_id
0,212


In [15]:
publishers.query('publisher_id == 212')

,publisher_id,publisher
211,212,Penguin Books


We identify the publisher that has released the greatest number of books with more than 50 pages - 'publisher_id' = 212, that corresponds to Penguin Books.

In [16]:
# Output the results for task: Identify the author with the highest average book rating (look only at books with 
# at least 50 ratings).
pd.io.sql.read_sql(query4, con = engine)

,author
0,J.K. Rowling/Mary GrandPré


We identify the author with the highest average book rating (only at books with at least 50 ratings). It's J.K. Rowling/Mary GrandPré.

In [17]:
# Output the results for task: Find the average number of text reviews among users who rated more than 50 books.
pd.io.sql.read_sql(query5, con = engine)

,Average review count
0,24.333333


We find the average number of text reviews among users who rated more than 50 books - 24 reviews.

## Conclusions

We would recommend that new product focus on books, released after year 2000, with a reviews from users who rated more than 50 books, and high ratings (>=4), paying special attention on books, published by Penguin Books. No need to say that J.K. Rowling is a must for every books app.